# Run Partial Matching Baseline

This notebook will run alignment algorithms on clean to modified audio

In [1]:
import numpy as np
import numba
import time
import librosa as lb
import multiprocessing
import subprocess
import pickle
import logging
import sys
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os.path
from pathlib import Path
from align_algs import *

In [2]:
hop_sec = 512 * 1 / 22050
n_cores = 8
downsample = 1

In [3]:
times = np.array([5, 10, 15, 20, 25, 30])
featdir1_root = Path('/home/cchang/ttmp/features/partial_match')
featdir2 = Path('/home/cchang/ttmp/features/clean')

In [4]:
# Set which cfg file we want to use for our tests
cfg_options = ['toy', 'small', 'medium', 'train_benchmark', 'test_benchmark']
cfg = cfg_options[1]

if cfg == 'toy':
    query_list = 'cfg_files/query.train_toy.list'
elif cfg == 'small':
    query_list = 'cfg_files/query.train_small.list'
elif cfg == 'medium':
    query_list = 'cfg_files/query.train_medium.list'
elif cfg == 'train_benchmark':
    query_list = 'cfg_files/query.train.list'
elif cfg == 'test_benchmark':
    query_list = 'cfg_files/query.test.list'

## Algorithms

In [5]:
# def align_batch(system, querylist, featdir1, featdir2, outdir, n_cores=8, downsample=1, verbose=False, **kwargs):
#     '''
#     Wrapper function for batch alignment that accepts a parameter for the baseline system number instead of parameters for steps and weights
#     '''
    
#     if system==1:
#         # Baseline 1: DTW with transitions (1,1), (1,2), (2,1) and weights 2, 3, 3
#         steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
#         weights = np.array([2,3,3])
#         inputs = alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose)
        
#     elif system==2:
#         # Baseline 2: DTW with transitions (1,1), (1,2), (2,1) and weights 1, 1, 1
#         steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
#         weights = np.array([1,1,1])
#         inputs = alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose)
        
#     elif system==3:
#         # Baseline 3: DTW with transitions (1,1), (1,0), (0,1) and weights 2, 1, 1
#         steps = np.array([1,1,1,0,0,1]).reshape((-1,2))
#         weights = np.array([2,1,1])
#         inputs = alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose)
        
#     elif system==4:
#         # Baseline 4: DTW with transitions (1,1), (1,0), (0,1) and weights 1, 1, 1
#         steps = np.array([1,1,1,0,0,1]).reshape((-1,2))
#         weights = np.array([1,1,1])
#         inputs = alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose)
        
#     elif system==5:
#         # Baseline 5: Subsequence DTW with (query, reference) transitions (1,1), (1,2), (2,1) and weights 1, 1, 2
#         steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
#         weights = np.array([1,1,2])
#         inputs = alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose, subseq=True)
        
#     elif system==6:
#         # Baseline 6: NWTW
#         gamma = 0.346
#         inputs = alignNW_batch(querylist, featdir1, featdir2, outdir, n_cores, downsample, gamma)
        
# #     elif system==7:
# #         # Baseline 7: Jump DTW with transitions (1,1), (1,0), (0,1) and weights 2, 1, 1 (same as 3)
# # #         steps = np.array([1,1,1,0,0,1]).reshape((-1,2))
# # #         weights = np.array([2,1,1])
# #         steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# #         weights = np.array([2,3,3])
# #         jumpweight = 1
# #         jumplocsFile = kwargs['jumplocsFile'] # TODO: Add check for keyword
# #         inputs = alignJumpDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, jumplocsFile, jumpweight, downsample, subseq = False, sr=22050, hop=512)
        
#     else:
#         logging.error('Unrecognized baseline ID (should be between 1 and 7) %s' % system)
#         sys.exit(1)
        
#     return

In [6]:
# def alignDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, downsample, verbose = False, subseq = False, librosa = True):
#     start = time.time()
#     outdir.mkdir(parents=True, exist_ok=True)
    
#     # prep inputs for parallelization
#     inputs = []
#     with open(querylist, 'r') as f:
#         for line in f:
#             parts = line.strip().split(' ')
#             assert len(parts) == 2
#             featfile1 = (featdir1 / parts[0]).with_suffix('.npy')
#             featfile2 = (featdir2 / parts[1]).with_suffix('.npy')
#             queryid = os.path.basename(parts[0]) + '__' + os.path.basename(parts[1])
#             outfile = (outdir / queryid).with_suffix('.pkl')
            
#             if os.path.exists(outfile):
#                 print(f"Skipping {outfile}")
#             else:
#                 inputs.append((featfile1, featfile2, steps, weights, downsample, subseq, outfile, verbose, librosa))

#     # process files in parallel
#     with multiprocessing.get_context("spawn").Pool(processes = n_cores) as pool:
#         pool.starmap(alignDTW, inputs)
#     print("Time to finish alignDTW: ", time.time() - start)
#     return

In [7]:
# @numba.jit(forceobj=True)
# def alignDTW(featfile1, featfile2, steps, weights, downsample, subseq = False, outfile = None, verbose = False, librosa = True):
#     '''
#     Aligns featfile1 and featfile2
    
#     Arguments:
#     subsequence -- if True, runs subsequenceDTW instead of DTW
#     verbose -- if True, prints statements for each file while processing
#     librosa -- if True, uses librosa's implementation of DTW
#     '''
#     if verbose:
#         print(outfile)
    
#     # Read in chroma features from feature files
#     F1 = np.load(featfile1) # 12 x N
#     F2 = np.load(featfile2) # 12 x M
    
#     if verbose:
#         print('Read chroma features')
#         print(F1)
#         print(F2)

#     # Make sure there is a valid path possible. If one file is over twice as long as the other one, 
#     # then no valid path is possible (assuming our steps only let us move a max of 2 spaces)
#     if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
#         if verbose:
#             print('Not valid')
#         if outfile:
#             pickle.dump(None, open(outfile, 'wb'))
#         return None
    
#     if verbose:
#         print('Checked if valid')
    
#     # For some reason, this calculation stalls for some pairs. Need to investigate more
#     C = 1 - F1[:,0::downsample].T @ F2[:,0::downsample] # cos distance metric
    
#     if verbose:
#         print('Calculated cost matrix if enabled')
    
#     # Run DTW algorithm
    
#     # This is Librosa's implementation
#     if librosa:
#         D, wp = lb.sequence.dtw(C=C, step_sizes_sigma = steps, weights_add = weights, subseq = subseq)
# #         D, wp = lb.sequence.dtw(X=F1, Y=F2, metric='cosine', step_sizes_sigma = steps, weights_add = weights, subseq = subseq)
#         wp = wp[::-1] # Need to reverse path for lb
    
#         # if N > M, Y can be a subsequence of X, librosa switches C to C transpose, so we want to switch the rows and columns of wp back
#         if subseq and (F1.shape[1] > F2.shape[1]):
#             wp = np.fliplr(wp)
            
#     else:
#     # This is our implementation. Currently needs cost matrix as input to run
#         optcost, wp = DTW(C, steps, weights, subseq = subseq)
#     if verbose:
#         print('Ran DTW')
    
#     # If output file is specified, save results
#     if outfile:
#         pickle.dump(wp, open(outfile, 'wb'))
#         if verbose:
#             print('Pickle dump')
#     else:
#         return wp

In [8]:
# @numba.njit()
# def DTW(C, steps, weights, subseq = False):
#     '''
#     Find the optimal subsequence path through cost matrix C.
    
#     Arguments:
#     C -- cost matrix of dimension (# query frames, # reference frames)
#     steps -- a numpy matrix specifying the allowable transitions.  It should be of
#             dimension (L, 2), where each row specifies (row step, col step)
#     weights -- a vector of size L specifying the multiplicative weights associated 
#                 with each of the allowable transitions
#     subsequence -- if True, runs subsequence DTW instead of regular DTW
                
#     Returns:
#     optcost -- the optimal subsequence path score
#     path -- a matrix with 2 columns specifying the optimal subsequence path.  Each row 
#             specifies the (row, col) coordinate.
#     '''
#     D = np.zeros(C.shape)
#     B = np.zeros(C.shape, dtype=np.int8)
#     if subseq:
#         D[0,:] = C[0,:]
#     else:
#         D[0,0] = C[0,0]
#     for row in range(1,C.shape[0]):
#         for col in range(C.shape[1]):
#             mincost = np.inf
#             minidx = -1
#             for stepidx, step in enumerate(steps):
#                 (rstep, cstep) = step
#                 prevrow = row - rstep
#                 prevcol = col - cstep
#                 if prevrow >= 0 and prevcol >= 0:
#                     pathcost = D[prevrow, prevcol] + C[row, col] * weights[stepidx]
#                     if pathcost < mincost:
#                         mincost = pathcost
#                         minidx = stepidx
#             D[row, col] = mincost
#             B[row, col] = minidx
            
#     if subseq:
#         optcost = np.min(D[-1,:])
#     else:
#         optcost = D[-1,-1]
    
#     path = backtrace(D, B, steps)
#     path.reverse()
#     path = np.array(path)
    
#     return optcost, path

In [9]:
# # @numba.njit()
# def backtrace(D, B, steps):
#     '''
#     Backtraces through the cumulative cost matrix D.
    
#     Arguments:
#     D -- cumulative cost matrix
#     B -- backtrace matrix
#     steps -- a numpy matrix specifying the allowable transitions.  It should be of
#             dimension (L, 2), where each row specifies (row step, col step)
    
#     Returns:
#     path -- a python list of (row, col) coordinates for the optimal path.
#     '''
#     # initialization
#     r = B.shape[0] - 1
#     c = B.shape[1] - 1
#     path = [[r, c]]
    
#     # backtrace
#     while r > 0:
#         step = steps[B[r, c]]
#         r = r - step[0]
#         c = c - step[1]
#         if r != B.shape[0] - 1:
#             path.append([r, c])
            
#     return path

In [10]:
from align_algs import *

## Calculate Alignments

In [6]:
# For DTW Baselines
folder_names = ['dtw233', 'dtw111']
for t in times:
    EXPERIMENTS_ROOT = '/home/cchang/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (cfg, t)
    
    for i in range(1, len(folder_names) + 1):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i-1])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
        inputs = align_batch(i, query_list, featdir1, featdir2, outdir, n_cores, downsample)

Time to finish alignDTW:  1308.8862309455872
Time to finish alignDTW:  1315.7620177268982
Time to finish alignDTW:  1325.4251580238342
Time to finish alignDTW:  1316.7038452625275
Time to finish alignDTW:  1318.403090953827
Time to finish alignDTW:  1326.2641677856445
Time to finish alignDTW:  1324.1620078086853
Time to finish alignDTW:  1322.9705564975739
Time to finish alignDTW:  1325.3922770023346
Time to finish alignDTW:  1328.6410856246948
Time to finish alignDTW:  1325.11865067482
Time to finish alignDTW:  1324.99085354805


In [ ]:
# For NWTW
folder_names = ['nwtw/gamma_0.8']
gamma = 0.8
for t in times[5:]:
    EXPERIMENTS_ROOT = '/home/cchang/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (cfg, t)
    
    for i in range(1):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
#         inputs = align_batch(i, query_list, featdir1, featdir2, outdir, n_cores, downsample)
        alignNW_batch(query_list, featdir1, featdir2, outdir, n_cores, downsample, gamma)

In [5]:
times = np.array([5, 10, 15, 20, 25, 30])
gamma = 20
beta = 10
steps = np.array([1,1,2,1,1,2]).reshape((-1,2))
weights = np.array([2,3,3])
folder_names = ['hstw/gamma_20_beta_10']
n_cores = 8

In [6]:
folder_names

['hstw/gamma_20_beta_10']

In [ ]:
# For HSTW
for t in times:
    EXPERIMENTS_ROOT = '/home/cchang/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (cfg, t)
    
    for i in range(len(folder_names)):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
        align_HPTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, gamma, beta, downsample)